In [47]:
import pandas as pd
import math
# price : 제품을 얻기 위해 지불하는 가격
# retail_price : 시장 또는 다른 상점/장소에서 유사한 물품에 대한 참조 가격. 
# currency_buyer : 현재 가격
# units_sold : 판매된 개수
# rating : 평균제품등급
# rating_count : 제품의 총 등급 개수
# badges_count : 제품 또는 판매자가 보유한 배지 수
# badge_local_product : 지역 제품이냐 아니냐 (1이면 지역제품)
# tags : 판매자가 붙인 꼬리표
# product_color : 제품 색
# countries_shipped_to : 이 제품이 배송되는 국가 수
# merchant_rating_count : 이 판매자의 등급 수
# merchant_rating : 상인의 등급.
# product_id : 제품 식별자
# urgency_text : 검색 결과의 일부 제품 위에 나타나는 텍스트 배너
#              - Réduction sur les achats en gros : 도매 구매 할인
#              - Quantité limitée : 제한 수량

summer_products=pd.read_csv("./summer-products-with-rating-and-performance_2020-08.csv")
summer_products_df=summer_products[["price","units_sold","rating","rating_count","merchant_rating_count","urgency_text"]]
summer_products_df.head()

,price,units_sold,rating,rating_count,merchant_rating_count,urgency_text
0,16.00,100,3.76,54,568,Quantité limitée !
1,8.00,20000,3.45,6135,17752,Quantité limitée !
2,8.00,100,3.57,14,295,Quantité limitée !
3,8.00,5000,4.03,579,23832,NaN
4,2.72,100,3.10,20,14482,Quantité limitée !


In [48]:
# 데이터 전처리
summer_products_df["urgency_text"]=summer_products_df["urgency_text"].fillna("-")
# drop_duplicates() : 중복 행이 제거 된 DataFrame을 반환한다.
urgency_text=summer_products_df["urgency_text"].drop_duplicates().to_numpy()

# urgency_text 데이터 라벨링 
encoder={}
for i,text in enumerate(urgency_text):
    encoder[text]=i

<ipython-input-48-7ed6ba8e8ed5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  summer_products_df["urgency_text"]=summer_products_df["urgency_text"].fillna("-")


In [49]:
summer_products_df["urgency_num"]=summer_products_df["urgency_text"].apply(lambda x:encoder[x])
summer_products_df=summer_products_df.drop("urgency_text",axis=1)

summer_products_df["units_sold"]=summer_products_df["units_sold"].apply(lambda x: math.sqrt(x)/10)

summer_products_df["merchant_rating_count"]=summer_products_df["merchant_rating_count"].apply(lambda x:math.sqrt(x)/10)

summer_products_df["rating_count"]=summer_products_df["rating_count"].apply(lambda x:math.sqrt(x)/5)

summer_products_df.head()

<ipython-input-49-9643d8855611>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  summer_products_df["urgency_num"]=summer_products_df["urgency_text"].apply(lambda x:encoder[x])


,price,units_sold,rating,rating_count,merchant_rating_count,urgency_num
0,16.00,1.000000,3.76,1.469694,2.383275,0
1,8.00,14.142136,3.45,15.665248,13.323663,0
2,8.00,1.000000,3.57,0.748331,1.717556,0
3,8.00,7.071068,4.03,4.812484,15.437616,1
4,2.72,1.000000,3.10,0.894427,12.034118,0


- SVM(Support Vector Machine)\
    : 이항 분류 문제를 위해 만들졌다
    - 클래스들이 선형경계에 의해 분류될 수 있는 데이터에만 사용가능
    - 결정 경계 : 클래스들을 구분하기 위한 선
    - 서포트 벡터 : 결정 경계 근처의 가장 가까운 포인터들
    - 마진(Margin) : 결정 경계와 서포트 벡터사이의 거리
    - 파라미터
        - C : 데이터 샘픔들이 다른 클래스에 놓이는 것을 허용하는 정도 
        - kernel : 주어진 데이터를 고차원 특징 공간으로 사상(결정 경계 정의 모형)\
        (linear, plotnormaial, RBF)
  
        - gamma : 결정 경계의 곡률을 결정
- SVM 종류
    - SVR(Support Vector Regression)
        - 회귀에 적용하는 방법
            - svm 회귀는 제한된 마진 오류(즉, 도로 밖의 샘플)안에서 도로 안에 가능한 한 많은 샘플이 들어가도록 학습한다
            - 도로 폭은 epsilon 하이퍼파라미터로 조절
    - SVC(Support Vector Classifiction)
        - 일정한 마진 오류 안에서, 두 클래스 간의 도로 폭이 가능한 한 최대가 되도록 한다\
        
        
https://analysis-flood.tistory.com/96 \
https://bskyvision.com/163 \
http://hleecaster.com/ml-svm-concept/#comment-785 

In [105]:
from sklearn.svm import SVR # 비선형 회귀 작업 
from sklearn.model_selection import train_test_split
import numpy as np

x=summer_products_df.to_numpy()[:,1:]
y=summer_products_df["price"].to_numpy()
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.2)
print(x.shape,y.shape)

# 모델 생성
svr=SVR(max_iter=1000)

# 모델 학습
svr.fit(X_train,y_train)

# 모델 예측
predict_y=svr.predict(X_test)

#  support vectors의 좌표 확인
# np.round(svr.support_vectors_,3)
for i,y in enumerate(predict_y):
    i=np.round(i,4)
    y=np.round(y,4)
    print("y_test :",y_test[i],"  ","predict : ",y)
#[{y_test[i]:y} for i,y in enumerate(predict_y)]

(1573, 5) (1573,)
y_test : 6.0    predict :  8.0571
y_test : 11.0    predict :  7.4616
y_test : 5.84    predict :  7.5268
y_test : 6.0    predict :  7.9124
y_test : 8.0    predict :  7.9002
y_test : 16.0    predict :  8.1956
y_test : 5.0    predict :  7.4224
y_test : 11.0    predict :  7.9296
y_test : 7.0    predict :  7.8312
y_test : 3.84    predict :  7.7656
y_test : 8.0    predict :  7.3425
y_test : 8.0    predict :  7.7949
y_test : 1.0    predict :  8.1248
y_test : 9.0    predict :  7.532
y_test : 14.0    predict :  8.0795
y_test : 6.0    predict :  7.957
y_test : 1.77    predict :  7.5174
y_test : 7.0    predict :  8.0947
y_test : 8.0    predict :  7.7764
y_test : 16.0    predict :  7.8736
y_test : 12.0    predict :  8.0047
y_test : 2.69    predict :  7.482
y_test : 8.0    predict :  8.8658
y_test : 8.0    predict :  7.4671
y_test : 11.0    predict :  7.988
y_test : 7.0    predict :  7.7001
y_test : 7.0    predict :  7.7269
y_test : 8.0    predict :  8.3407
y_test : 7.0    predict

In [88]:
from sklearn.svm import SVC

x=summer_products_df.to_numpy()[:,:-1]
y=summer_products_df["urgency_num"].to_numpy()
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.2)
print(x.shape,y.shape)

# 모델 생성
scv=SVC(kernel="sigmoid",max_iter=100)

# 모델 학습
scv.fit(X_train,y_train)

# 모델 예측
predict_y=scv.predict(X_test)

# support vectors의 좌표 확인
# np.round(scv.support_vectors_,3)
for i,y in enumerate(predict_y):
    print(i,y)
#[{y_test[i]:y} for i,y in enumerate(predict_y)]

(1573, 5) (1573,)
0 1
1 1
2 1
3 1
4 1
5 0
6 0
7 0
8 1
9 1
10 1
11 0
12 0
13 1
14 0
15 0
16 1
17 0
18 1
19 0
20 0
21 0
22 0
23 0
24 0
25 1
26 1
27 0
28 0
29 0
30 0
31 1
32 1
33 0
34 0
35 1
36 1
37 0
38 0
39 0
40 0
41 1
42 1
43 0
44 1
45 0
46 0
47 1
48 0
49 0
50 0
51 0
52 0
53 1
54 0
55 0
56 1
57 0
58 0
59 0
60 1
61 0
62 1
63 0
64 1
65 1
66 0
67 0
68 1
69 0
70 1
71 1
72 1
73 1
74 0
75 0
76 1
77 0
78 0
79 1
80 0
81 0
82 0
83 1
84 1
85 1
86 1
87 1
88 0
89 1
90 0
91 1
92 1
93 0
94 1
95 0
96 0
97 1
98 1
99 0
100 0
101 1
102 0
103 0
104 0
105 1
106 1
107 1
108 1
109 1
110 0
111 1
112 1
113 0
114 0
115 0
116 1
117 0
118 0
119 0
120 0
121 1
122 0
123 0
124 0
125 1
126 0
127 1
128 0
129 1
130 0
131 0
132 0
133 1
134 0
135 0
136 0
137 1
138 1
139 0
140 1
141 1
142 0
143 1
144 0
145 1
146 1
147 1
148 1
149 1
150 1
151 1
152 1
153 1
154 1
155 0
156 1
157 1
158 0
159 0
160 0
161 0
162 0
163 1
164 0
165 0
166 1
167 0
168 0
169 0
170 1
171 1
172 0
173 1
174 0
175 0
176 1
177 1
178 1
179 0
180 0
181 1


C:\Users\hyebin\anaconda3\lib\site-packages\sklearn\svm\_base.py:246: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
